In [1]:
import pandas as pd
import os

from google.cloud import bigquery
from datetime import datetime, date
import re

pd.set_option('display.max_rows', None)

from pathlib import Path 

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"C:\Users\Greg\OneDrive - Consumer Edge Research LLC\Desktop\Python\gcp-key-2.json"
client = bigquery.Client()

now = datetime.now()
day = now.strftime("%d")
month = now.strftime("%m")
year = now.strftime("%Y")

In [3]:
#all mids that are tagged to 2 or more main brands in US
us_dupe_mids = 'WITH dupe_main_mids AS (SELECT mid,COUNT(main_brand_name) AS brand_count FROM (SELECT DISTINCT mid,b.brand_name AS main_brand_name,b2.brand_name AS distribution_brand_name,b3.brand_name AS payment_brand_name FROM `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_enhanced` m LEFT JOIN `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc ON mc.mcc = m.mcc LEFT JOIN `beta-omega-alpha-theta.reference.mcc_mapping` mcc ON mcc.mcc = m.mcc LEFT JOIN `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_match_brand_ats` mm ON mm.merchid = m.merchid LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b ON b.brand_id = mm.main_brand_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b2 ON b2.brand_id = mm.distribution_brand_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b3 ON b3.brand_id = mm.payment_brand_id WHERE b.brand_name IS NOT NULL) GROUP BY mid HAVING brand_count > 1 ORDER BY brand_count DESC), dupe_dist_mids AS (SELECT mid,COUNT(main_brand_name) AS brand_count FROM (SELECT DISTINCT m.mid, b.brand_name AS main_brand_name,b2.brand_name AS distribution_brand_name, b3.brand_name AS payment_brand_name FROM `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_enhanced` m LEFT JOIN `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc ON mc.mcc = m.mcc LEFT JOIN `beta-omega-alpha-theta.reference.mcc_mapping` mcc ON mcc.mcc = m.mcc LEFT JOIN `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_match_brand_ats` mm ON mm.merchid = m.merchid LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b ON b.brand_id = mm.main_brand_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b2 ON b2.brand_id = mm.distribution_brand_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b3 ON b3.brand_id = mm.payment_brand_id WHERE b2.brand_name IS NOT NULL) GROUP BY mid HAVING brand_count > 1 ORDER BY brand_count DESC), dupe_payment_mids AS (SELECT payment_brand_tagged_spend.mid, COUNT(payment_brand_tagged_spend.main_brand_name) AS brand_count FROM (SELECT DISTINCT m.mid, b.brand_name AS main_brand_name, b2.brand_name AS distribution_brand_name, b3.brand_name AS payment_brand_name FROM `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_enhanced` m LEFT JOIN `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc ON mc.mcc = m.mcc LEFT JOIN `beta-omega-alpha-theta.reference.mcc_mapping` mcc ON mcc.mcc = m.mcc LEFT JOIN `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_match_brand_ats` mm ON mm.merchid = m.merchid LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b ON b.brand_id = mm.main_brand_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b2 ON b2.brand_id = mm.distribution_brand_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b3 ON b3.brand_id = mm.payment_brand_id WHERE b3.brand_name IS NOT NULL) AS payment_brand_tagged_spend GROUP BY payment_brand_tagged_spend.mid HAVING brand_count > 1 ORDER BY brand_count DESC) SELECT mid FROM dupe_payment_mids UNION DISTINCT SELECT mid FROM dupe_dist_mids UNION DISTINCT SELECT mid FROM dupe_main_mids'
us_dupe_mids_df = client.query(us_dupe_mids).to_dataframe()

#all mids that are tagged to 2 or more main brands in UK
uk_dupe_mids = 'with dupe_main_mids as (SELECT main_brand_tagged_spend.mid, COUNT(main_brand_tagged_spend.main_brand_name) as brand_count FROM (SELECT distinct m.mid, b.brand_name as main_brand_name, b2.brand_name as distribution_brand_name, b3.brand_name as payment_brand_name FROM `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc left join `beta-omega-alpha-theta.reference.mcc_mapping` mcc on mcc.mcc = m.mcc left join `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_match_ats` mm on mm.merchid = m.merchid left join `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_match_brand_ats` mmm on m.merchid = mmm.merchid left join `beta-omega-alpha-theta.ground_truth.brand_detail` b on b.brand_id = mm.main_brand_id left join `beta-omega-alpha-theta.ground_truth.brand_detail` b2 on b2.brand_id = mm.distribution_brand_id left join `beta-omega-alpha-theta.ground_truth.brand_detail` b3 on b3.brand_id = mm.payment_brand_id WHERE b.brand_name IS NOT NULL) as main_brand_tagged_spend GROUP BY main_brand_tagged_spend.mid HAVING brand_count > 1 ORDER BY brand_count DESC ), dupe_dist_mids as ( SELECT dist_brand_tagged_spend.mid, COUNT(dist_brand_tagged_spend.main_brand_name) as brand_count FROM (SELECT distinct m.mid, b.brand_name as main_brand_name, b2.brand_name as distribution_brand_name, b3.brand_name as payment_brand_name FROM `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc left join `beta-omega-alpha-theta.reference.mcc_mapping` mcc on mcc.mcc = m.mcc left join `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_match_ats` mm on m.merchid = mm.merchid left join `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_match_brand_ats` mmm on m.merchid = mmm.merchid left join `beta-omega-alpha-theta.ground_truth.brand_detail` b on b.brand_id = mm.main_brand_id left join `beta-omega-alpha-theta.ground_truth.brand_detail` b2 on b2.brand_id = mm.distribution_brand_id left join `beta-omega-alpha-theta.ground_truth.brand_detail` b3 on b3.brand_id = mm.payment_brand_id WHERE b2.brand_name IS NOT NULL) as dist_brand_tagged_spend GROUP BY dist_brand_tagged_spend.mid HAVING brand_count > 1 ORDER BY brand_count DESC), dupe_payment_mids as ( SELECT payment_brand_tagged_spend.mid, COUNT(payment_brand_tagged_spend.main_brand_name) as brand_count FROM (SELECT distinct m.mid, b.brand_name as main_brand_name, b2.brand_name as distribution_brand_name, b3.brand_name as payment_brand_name FROM `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc left join `beta-omega-alpha-theta.reference.mcc_mapping` mcc on mcc.mcc = m.mcc left join `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_match_brand_ats` mm on mm.merchid = m.merchid left join `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_match_brand_ats` mmm on m.merchid = mmm.merchid left join `beta-omega-alpha-theta.ground_truth.brand_detail` b on b.brand_id = mm.main_brand_id left join `beta-omega-alpha-theta.ground_truth.brand_detail` b2 on b2.brand_id = mm.distribution_brand_id left join `beta-omega-alpha-theta.ground_truth.brand_detail` b3 on b3.brand_id = mm.payment_brand_id WHERE b3.brand_name IS NOT NULL) as payment_brand_tagged_spend GROUP BY payment_brand_tagged_spend.mid HAVING brand_count > 1 ORDER BY brand_count DESC) select mid from dupe_payment_mids UNION DISTINCT select mid from dupe_dist_mids UNION DISTINCT select mid from dupe_main_mids'
uk_dupe_mids_df = client.query(uk_dupe_mids).to_dataframe()

In [4]:
#tagged spend based on brand_name variable and spend_amount threshold
brand_name = "'%MAIN%EVENT%ENT%'"
spend_amount = '1000'
region = 'US' #'US' or 'UK'

filepath = Path('C:/Users/Greg/OneDrive - Consumer Edge Research LLC/Desktop/Python/Exports/' + brand_name + '_Mid_Check_' + month +'_' + day + '.csv')

In [5]:
if region == 'US':
    brand_tagging = 'SELECT * FROM (SELECT m.mid,LEFT(m.merchdesc,15) as merchdesc,m.mcc,m.state,m.city,m.zip,m.avg_spend_amount,m.spend_amount,m.first_trans_date,m.last_trans_date,m.top_state_abbr,m.pct_state_trans,  main_pattern_match_id,b.brand_name as main_brand_name,b2.brand_name as distribution_brand_name,b3.brand_name as payment_brand_name,mmm.subchannel,m.merchid,m.top_cobrid FROM `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc LEFT JOIN `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_match_brand_ats` mm on m.merchid = mm.merchid LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b on mm.main_brand_id = b.brand_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b2 on mm.distribution_brand_id = b2.brand_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b3 on mm.payment_brand_id = b3.brand_id LEFT JOIN `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_match_subchannel_ats` mmm on m.merchid = mmm.merchid) where spend_amount > ' + spend_amount + ' and main_brand_name LIKE ' + brand_name
    brand_tagging_df = client.query(brand_tagging).to_dataframe()

    #unique tagged brand mids
    brand_mids = brand_tagging_df['mid'].drop_duplicates()
    
    #set difference of unique tagged brand mids and known shared US mids
    unique_brand_mids = list(set(brand_mids) - set(us_dupe_mids_df['mid']))

    mid_sql = '''SELECT * FROM (SELECT  b.brand_name as main_brand_name, m.mid, m.merchdesc, m.mcc, mc.mcc_category, m.city, m.state, m.zip, m.avg_spend_amount, m.spend_amount, m.first_trans_date, m.last_trans_date, b2.brand_name as distribution_brand_name, b3.brand_name as payment_brand_name, m.merchid FROM `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc LEFT JOIN `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_match_brand_ats` mm on m.merchid = mm.merchid LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b on mm.main_brand_id = b.brand_id  LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b2 on mm.distribution_brand_id = b2.brand_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b3 on mm.payment_brand_id = b3.brand_id) where main_brand_name IS NULL and mid IN UNNEST(%s)''' %(unique_brand_mids)
    mid_sql = mid_sql + ' and spend_amount > ' + spend_amount + ' ORDER BY spend_amount DESC'
    untagged_spend = client.query(mid_sql).to_dataframe()

if region == 'UK':
    brand_tagging = 'SELECT m.merchid,m.mid,m.merchdesc,m.mcc,m.location,m.postcode,round(m.avg_spend_amount, 0) as avg_spend_amount,round(m.stddev_spend_amount, 0) as stddev_spend_amount,round(m.spend_amount, 0) as spend_amount,m.card_count,m.trans_count,mm.channel,m.first_trans_date,m.last_trans_date,m.day_count,m.top_cardtype,b.brand_name as main_brand_name,b2.brand_name as distribution_brand_name,b3.brand_name as payment_brand_name,kmm.brand_source,b4.brand_name as kmm_main_brand_name,mmm.main_pattern_match_id FROM `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_enhanced` m LEFT JOIN `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_match_ats` mm on m.merchid = mm.merchid LEFT JOIN `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_match_brand_ats` mmm on m.merchid = mmm.merchid LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b on mm.main_brand_id = b.brand_id LEFT JOIN `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_match` kmm on m.merchid = kmm.merchid LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b2 on b2.brand_id = mm.distribution_brand_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b3 on b3.brand_id = mm.payment_brand_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b4 on b4.brand_id = kmm.main_brand_id where spend_amount > ' + spend_amount + ' and b.brand_name LIKE ' + brand_name
    brand_tagging_df = client.query(brand_tagging).to_dataframe()

    #unique tagged brand mids
    brand_mids = brand_tagging_df['mid'].drop_duplicates()
    
    #set difference of unique tagged brand mids and known shared UK mids
    unique_brand_mids = list(set(brand_mids) - set(uk_dupe_mids_df['mid']))

    mid_sql = '''SELECT DISTINCT m.mid, m.merchdesc, m.mcc, mcc.mcc_category, round(m.spend_amount, 0) as spend_amount, FROM `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_enhanced` m LEFT JOIN `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_match_ats` mm on m.merchid = mm.merchid left join `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_match_brand_ats` mmm on mmm.merchid = m.merchid left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc left join `beta-omega-alpha-theta.reference.mcc_mapping` mcc on mcc.mcc = m.mcc left join `beta-omega-alpha-theta.ground_truth.brand_detail` b on b.brand_id = mm.main_brand_id left join `beta-omega-alpha-theta.ground_truth.brand_detail` b2 on b2.brand_id = mm.distribution_brand_id left join `beta-omega-alpha-theta.ground_truth.brand_detail` b3 on b3.brand_id = mm.payment_brand_id where b.brand_name IS NULL and mid IN UNNEST(%s)'''%(unique_brand_mids)
    mid_sql = mid_sql + ' and spend_amount > ' + spend_amount + ' ORDER BY spend_amount DESC'
    untagged_spend = client.query(mid_sql).to_dataframe()

In [6]:
untagged_spend['mid'] = untagged_spend['mid'].str.lstrip("0")
untagged_spend_mids = untagged_spend['mid'].drop_duplicates()

mid_regex = '^0*('
for i, line in enumerate(untagged_spend_mids):
    if i < len(untagged_spend_mids)-1:
        regex_i = line + '|'
        mid_regex += regex_i
    else:
        regex_i = line + ')$'
        mid_regex += regex_i
mid_regex

'^0*(8788290330733|517530860111160|510165370100455|844219156883|479338001911773)$'

In [7]:
if region == 'US':
    sql_qa = "SELECT mid, merchdesc, mcc, mcc_category, spend_amount, first_trans_date, main_brand_name FROM (SELECT  b.brand_name as main_brand_name, m.mid, m.merchdesc, m.mcc, mc.mcc_category, m.city, m.state, m.zip, m.avg_spend_amount, m.spend_amount, m.first_trans_date, m.last_trans_date, b2.brand_name as distribution_brand_name, b3.brand_name as payment_brand_name, m.merchid FROM `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc LEFT JOIN `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_match_brand_ats` mm on m.merchid = mm.merchid LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b on mm.main_brand_id = b.brand_id  LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b2 on mm.distribution_brand_id = b2.brand_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b3 on mm.payment_brand_id = b3.brand_id) where regexp_contains(mid, '" + mid_regex + "') and spend_amount > " + spend_amount + " and main_brand_name IS NULL ORDER BY spend_amount DESC"
    sql_qa_df = client.query(sql_qa).to_dataframe()
    
if region == 'UK':
    sql_qa = "SELECT DISTINCT m.mid, m.merchdesc, m.mcc, mcc.mcc_category, round(m.spend_amount, 0) as spend_amount, round(m.avg_spend_amount, 0) as avg_spend_amount, b.brand_name FROM `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc left join `beta-omega-alpha-theta.reference.mcc_mapping` mcc on mcc.mcc = m.mcc left join `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_match_ats` mm on mm.merchid = m.merchid left join `beta-omega-alpha-theta.apollo_uk_ats_ml.mid_match_brand_ats` mmm on mmm.merchid = m.merchid left join `beta-omega-alpha-theta.ground_truth.brand_detail` b on b.brand_id = mm.main_brand_id left join `beta-omega-alpha-theta.ground_truth.brand_detail` b2 on b2.brand_id = mm.distribution_brand_id left join `beta-omega-alpha-theta.ground_truth.brand_detail` b3 on b3.brand_id = mm.payment_brand_id where regexp_contains(mid, '" + mid_regex + "') and spend_amount > " + spend_amount + " ORDER BY spend_amount DESC"
    sql_qa_df = client.query(sql_qa).to_dataframe()

In [8]:
#Untagged merchdesc using shared mids
sql_qa_df.head(200)

,mid,merchdesc,mcc,mcc_category,spend_amount,first_trans_date,main_brand_name
0,008788290330733,EVENT ENTERTAINMENT GR,7922,Theatrical Producers (Eexcept Motion Pictures)...,612084.380000,2014-01-02,None
1,517530860111160,MAIN EVENT SPORTS,5813,"Drinking Places (Alcoholic Beverages) Bars, Ta...",128042.730037,2022-03-29,None
2,008788290330733,EVENT ENTERTAINMENT GR,7922,Theatrical Producers (Eexcept Motion Pictures)...,106237.450000,2014-01-24,None
3,517530860111160,MAIN EVENT SPORTS GRILL,5812,Eating Places and Restaurants,68313.480000,2016-05-16,None
4,510165370100455,MAIN EVENT SPORTS,5812,Eating Places and Restaurants,50683.500000,2022-03-30,None
5,000844219156883,G&T ENTERTAINMENT INC,5812,Eating Places and Restaurants,33017.060000,2019-11-06,None
6,008788290330733,EVENT ENTERTAINM,7922,Theatrical Producers (Eexcept Motion Pictures)...,32935.800000,2014-01-03,None
7,517530860111160,MAIN EVENT SPORTS GRILL,5813,"Drinking Places (Alcoholic Beverages) Bars, Ta...",32740.479999,2021-07-22,None
8,844219156883,G&T ENTERTAINMENT,5812,Eating Places and Restaurants,31159.220000,2022-03-31,None
9,510165370100455,MAIN EVENT SPORTS BAR & E,5812,Eating Places and Restaurants,12691.900000,2015-09-21,None


In [9]:
mcc = '(' + "'5812'," + "'7999'" + ')'

brand_merchdesc = brand_tagging_df['merchdesc'].drop_duplicates().dropna()
unique_brand_merchdesc = list(brand_merchdesc)

merchdesc_sql = '''SELECT * FROM (SELECT  b.brand_name as main_brand_name, m.merchdesc, m.mcc, mc.mcc_category,m.avg_spend_amount, m.spend_amount FROM `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc LEFT JOIN `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_match_brand_ats` mm on m.merchid = mm.merchid LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b on mm.main_brand_id = b.brand_id  LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b2 on mm.distribution_brand_id = b2.brand_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b3 on mm.payment_brand_id = b3.brand_id) where main_brand_name IS NULL and merchdesc IN UNNEST(%s)''' %(unique_brand_merchdesc)
merchdesc_sql += ' and spend_amount > ' + spend_amount + ' and mcc IN ' + mcc + ' ORDER BY spend_amount DESC'

merchdesc_df = client.query(merchdesc_sql).to_dataframe()

In [10]:
#Similar untagged merchdesc (Will commonly be due to maroon missing mids pre 2022)
merchdesc_df

,main_brand_name,merchdesc,mcc,mcc_category,avg_spend_amount,spend_amount
0,None,,5812,Eating Places and Restaurants,53.024966,4.446143e+06
1,None,,5812,Eating Places and Restaurants,39.121782,2.359982e+06
2,None,THE SUMMIT,5812,Eating Places and Restaurants,38.613454,1.825219e+06
3,None,,5812,Eating Places and Restaurants,37.675876,1.416048e+06
4,None,,7999,Recreation Services (Not Elsewhere Classified),109.551371,1.213391e+06
5,None,,5812,Eating Places and Restaurants,46.544284,1.151785e+06
6,None,,5812,Eating Places and Restaurants,39.695759,8.965684e+05
7,None,,5812,Eating Places and Restaurants,48.231761,6.138939e+05
8,None,,5812,Eating Places and Restaurants,45.796306,5.452966e+05
9,None,,5812,Eating Places and Restaurants,53.378705,5.150511e+05
